<h2>Dados de voos com origem ou destino no Brasil de março/24</h2>

Dados da ANAC

In [116]:
import os
import pandas as pd
import numpy as np
import datetime

pasta = r"C:\Users\bserpellone\Desktop\Python\Dataset\anac"

registros = "VRA_2024_03.csv"
dAeroportos = "glossario_de_aerodromo.xls"
dViacoes = "glossario_de_empresas_aereas.xls"

df = pd.read_csv(os.path.join(pasta,registros),delimiter=';')
df_aeros = pd.read_excel(os.path.join(pasta,dAeroportos),header=3)
df_aeros.dropna(axis=1,how='all',inplace=True)

df_viacoes = pd.read_excel(os.path.join(pasta,dViacoes),header=3)
df_viacoes.dropna(axis=1,how='all',inplace=True)

df_merged = df.merge(df_viacoes,how='left',left_on='Sigla ICAO Empresa Aérea',right_on='Sigla OACI')
df_merged = df_merged.merge(df_aeros,left_on='Sigla ICAO Aeroporto Origem',right_on='Sigla OACI',how='left',suffixes=('_ori','_dest'))
df_merged = df_merged.merge(df_aeros,left_on='Sigla ICAO Aeroporto Destino',right_on='Sigla OACI',how='left',suffixes=('_Origem','_Destino'))

<h4>Quantas linhas/registros de voos tem na base?</h4>

In [35]:
df_selecao = df_merged[['Chegada Real','Situação Voo','Nome Empresas','Nacional ou Estrangeira','Partida Prevista','Partida Real','Cidade_Origem','UF_Origem', 'País_Origem','Continente_Origem','Cidade_Destino', 'UF_Destino', 'País_Destino','Continente_Destino']]
df_selecao.shape[0]

81560

Primeiras linhas da tabela:

In [95]:
df_selecao.head(5)

,Chegada Real,Situação Voo,Nome Empresas,Nacional ou Estrangeira,Partida Prevista,Partida Real,Cidade_Origem,UF_Origem,País_Origem,Continente_Origem,Cidade_Destino,UF_Destino,País_Destino,Continente_Destino,Diferença_segundos,Status_voo
0,02/03/2024 08:18,REALIZADO,AMERICAN AIRLINES INC,ESTRANGEIRA,2024-01-03 23:55:00,2024-01-03 23:48:00,RIO DE JANEIRO,RJ,BRASIL,AMERICA DO SUL,MIAMI,NaN,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,-420.0,De 5 a 20min adiantado
1,01/03/2024 11:42,REALIZADO,AMERICAN AIRLINES INC,ESTRANGEIRA,2024-01-03 23:55:00,2024-01-03 03:38:00,MIAMI,NaN,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,RIO DE JANEIRO,RJ,BRASIL,AMERICA DO SUL,-73020.0,20min+ adiantado
2,01/03/2024 08:11,REALIZADO,AMERICAN AIRLINES INC,ESTRANGEIRA,2024-01-03 00:55:00,2024-01-03 00:09:00,GUARULHOS,SP,BRASIL,AMERICA DO SUL,MIAMI,NaN,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,-2760.0,20min+ adiantado
3,02/03/2024 06:03,REALIZADO,AMERICAN AIRLINES INC,ESTRANGEIRA,2024-01-03 20:50:00,2024-01-03 21:58:00,MIAMI,NaN,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,GUARULHOS,SP,BRASIL,AMERICA DO SUL,4080.0,20min+ atrasado
4,02/03/2024 09:06,REALIZADO,AMERICAN AIRLINES INC,ESTRANGEIRA,2024-01-03 22:30:00,2024-01-03 22:34:00,GUARULHOS,SP,BRASIL,AMERICA DO SUL,MIAMI,NaN,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,240.0,Pontual


<h4>Qual a proporção de voos domésticos e estrangeiros?</h4>

In [36]:
df_selecao['Nacional ou Estrangeira'].value_counts(normalize=True).round(2)

Nacional ou Estrangeira
BRASILEIRA       0.88
ESTRANGEIRA      0.12
NÃO INFORMADO    0.00
Name: proportion, dtype: float64

In [97]:
df_selecao['Cidade_Origem'][df_selecao['Nacional ou Estrangeira']=='BRASILEIRA'].value_counts(normalize=True).round(2).head(5)

Cidade_Origem
GUARULHOS         0.13
SÃO PAULO         0.12
RIO DE JANEIRO    0.07
CAMPINAS          0.07
BRASÍLIA          0.07
Name: proportion, dtype: float64

<h4>Quais as principais cidades origem dos voos?</h4>

In [37]:
df_selecao['Cidade_Origem'].value_counts(normalize=True).round(2).head(10)

Cidade_Origem
GUARULHOS               0.14
SÃO PAULO               0.10
RIO DE JANEIRO          0.08
CAMPINAS                0.06
BRASÍLIA                0.06
CONFINS                 0.05
RECIFE                  0.04
PORTO ALEGRE            0.03
SÃO JOSÉ DOS PINHAIS    0.03
SALVADOR                0.03
Name: proportion, dtype: float64

<h4>Quais as principais cidades destino dos voos?</h4>

In [38]:
df_selecao['Cidade_Destino'].value_counts(normalize=True).round(2).head(10)

Cidade_Destino
GUARULHOS               0.14
SÃO PAULO               0.10
RIO DE JANEIRO          0.08
CAMPINAS                0.07
BRASÍLIA                0.06
CONFINS                 0.05
RECIFE                  0.04
PORTO ALEGRE            0.03
SÃO JOSÉ DOS PINHAIS    0.03
SALVADOR                0.03
Name: proportion, dtype: float64

<h4>Quais foram as 5 horas mais frequentes de voos pousando no Brasil (seja doméstico ou vindo do exterior)?</h4>

In [100]:
df_selecao['Chegada Real'].str[11:13][df_selecao['UF_Destino'].notna()].value_counts(normalize=True).round(3).head(5)

Chegada Real
16    0.064
11    0.063
19    0.059
10    0.058
07    0.056
Name: proportion, dtype: float64

Quais foram as 5 horas mais frequentes de voos decolando no Brasil (seja doméstico ou indo para o exterior)?

In [101]:
df_selecao['Chegada Real'].str[11:13][df_selecao['UF_Origem'].notna()].value_counts(normalize=True).round(3).head(5)

Chegada Real
11    0.065
16    0.060
10    0.059
13    0.058
07    0.057
Name: proportion, dtype: float64

<h4>Qual é a proporção de destinos por continente em voos internacionais que saíram do Brasil?</h4>

In [40]:
df_selecao['Continente_Destino'][df_selecao['Nacional ou Estrangeira']=='ESTRANGEIRA'].value_counts(normalize=True).round(2)

Continente_Destino
AMERICA DO SUL      0.69
EUROPA              0.14
AMERICA DO NORTE    0.09
AMERICA CENTRAL     0.05
ASIA                0.02
AFRICA              0.02
Name: proportion, dtype: float64

<h4> Quais os principais destinos internacionais (países)?</h4>

In [41]:
df_selecao['País_Destino'][df_selecao['País_Destino']!='BRASIL'].value_counts(normalize=True).round(2).head(5)

País_Destino
ARGENTINA                    0.21
ESTADOS UNIDOS DA AMERICA    0.14
CHILE                        0.14
PORTUGAL                     0.06
COLOMBIA                     0.05
Name: proportion, dtype: float64

<h4>Quais os principais destinos internacionais (cidades)?</h4>

In [42]:
df_selecao.loc[:,'Cidade_Destino'] = df_selecao['Cidade_Destino'].replace("BUENOS AIRES/AEROPARQUE","BUENOS AIRES")
df_selecao['Cidade_Destino'][df_selecao['País_Destino']!='BRASIL'].value_counts(normalize=True).round(2).head(10)

Cidade_Destino
BUENOS AIRES         0.19
SANTIAGO DO CHILE    0.14
MIAMI                0.06
LISBOA               0.06
PANAMA               0.05
BOGOTA               0.05
MONTEVIDEU           0.04
LIMA                 0.03
MADRID               0.03
TABABELA             0.02
Name: proportion, dtype: float64

<h4>Como é a proporção de viagens por aviação?</h4>

In [43]:
df_selecao['Nome Empresas'].value_counts(normalize=True).round(2).head(5)

Nome Empresas
AZUL                     0.34
TAM                      0.29
GOL                      0.22
PASSAREDO                0.02
AEROLINEAS ARGENTINAS    0.01
Name: proportion, dtype: float64

<h4>Como é a proporção de viagens por aviação estrangeira?</h4>

In [44]:
df_selecao['Nome Empresas'][df_selecao['Nacional ou Estrangeira']=="ESTRANGEIRA"].value_counts(normalize=True).round(2).head(5)

Nome Empresas
AEROLINEAS ARGENTINAS                  0.13
LAN CHILE                              0.09
TAP AIR PORTUGAL                       0.09
COPA -COMPANIA PANAMENA DE AVIACION    0.07
QATAR AIRWAYS                          0.06
Name: proportion, dtype: float64

<h4> Houve outras aviações fora Azul, Tam, Gol e Passaredo em voos nacionais?</h4>

In [45]:
df_selecao['Nome Empresas'][df_selecao['Nacional ou Estrangeira']=="BRASILEIRA"].value_counts(normalize=True).round(2)

Nome Empresas
AZUL                 0.38
TAM                  0.32
GOL                  0.25
PASSAREDO            0.02
MAP LINHAS AEREAS    0.01
ABSA                 0.01
TOTAL                0.00
ABAETE               0.00
Name: proportion, dtype: float64

<h4>Como foi o status dos voos em relação a voos nacionais e internacionais?</h4>

In [46]:
df_selecao.groupby('Nacional ou Estrangeira')['Situação Voo'].value_counts(normalize=True).round(2)

Nacional ou Estrangeira  Situação Voo 
BRASILEIRA               REALIZADO        0.97
                         CANCELADO        0.03
ESTRANGEIRA              REALIZADO        0.83
                         CANCELADO        0.17
                         NÃO INFORMADO    0.00
NÃO INFORMADO            REALIZADO        0.75
                         CANCELADO        0.25
Name: proportion, dtype: float64

In [127]:
df['Situação Voo'][df_selecao['Nacional ou Estrangeira']=='ESTRANGEIRA'].value_counts()

Situação Voo
REALIZADO        7396
CANCELADO        1462
NÃO INFORMADO       2
Name: count, dtype: int64

<h4>Qual a proporção de voos atrasados?</h4>

Valores da coluna situação voo:

In [66]:
df_selecao['Situação Voo'].drop_duplicates()

0            REALIZADO
10           CANCELADO
30918    NÃO INFORMADO
Name: Situação Voo, dtype: object

In [63]:
df_selecao.head(3)

,Chegada Real,Situação Voo,Nome Empresas,Nacional ou Estrangeira,Partida Prevista,Partida Real,Cidade_Origem,UF_Origem,País_Origem,Continente_Origem,Cidade_Destino,UF_Destino,País_Destino,Continente_Destino
0,02/03/2024 08:18,REALIZADO,AMERICAN AIRLINES INC,ESTRANGEIRA,2024-01-03 23:55:00,2024-01-03 23:48:00,RIO DE JANEIRO,RJ,BRASIL,AMERICA DO SUL,MIAMI,NaN,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE
1,01/03/2024 11:42,REALIZADO,AMERICAN AIRLINES INC,ESTRANGEIRA,2024-01-03 23:55:00,2024-01-03 03:38:00,MIAMI,NaN,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE,RIO DE JANEIRO,RJ,BRASIL,AMERICA DO SUL
2,01/03/2024 08:11,REALIZADO,AMERICAN AIRLINES INC,ESTRANGEIRA,2024-01-03 00:55:00,2024-01-03 00:09:00,GUARULHOS,SP,BRASIL,AMERICA DO SUL,MIAMI,NaN,ESTADOS UNIDOS DA AMERICA,AMERICA DO NORTE


Como não tem coluna indicado se o voo atrasou, vamos calcular operando as colunas partida real x partida prevista:

In [78]:
df_status = df_selecao.copy()

Nessa análise quero manter apenas os voos realizados

In [85]:
df_status = df_status[df_status['Situação Voo'] == 'REALIZADO']

Excluindo linhas sem informação de partida real ou partida prevista:

In [86]:
df_status[['Partida Real','Partida Prevista']].dropna(how='any')

,Partida Real,Partida Prevista
0,2024-01-03 23:48:00,2024-01-03 23:55:00
1,2024-01-03 03:38:00,2024-01-03 23:55:00
2,2024-01-03 00:09:00,2024-01-03 00:55:00
3,2024-01-03 21:58:00,2024-01-03 20:50:00
4,2024-01-03 22:34:00,2024-01-03 22:30:00
...,...,...
81494,2024-12-03 17:11:00,2024-12-03 17:35:00
81495,2024-12-03 06:15:00,2024-12-03 06:40:00
81496,2024-12-03 10:20:00,2024-12-03 10:30:00
81551,2024-07-03 13:14:00,2024-07-03 13:30:00


In [87]:
df_status[['Partida Real','Partida Prevista']].head(5)

,Partida Real,Partida Prevista
0,2024-01-03 23:48:00,2024-01-03 23:55:00
1,2024-01-03 03:38:00,2024-01-03 23:55:00
2,2024-01-03 00:09:00,2024-01-03 00:55:00
3,2024-01-03 21:58:00,2024-01-03 20:50:00
4,2024-01-03 22:34:00,2024-01-03 22:30:00


Abaixo calculamos a diferença, em segundos, entre as 2 colunas

In [107]:
df_status.loc[:,'Diferença_segundos'] = (pd.to_datetime(df_status['Partida Real'], errors='coerce') - pd.to_datetime(df_status['Partida Prevista'], errors='coerce')).dt.total_seconds()
df_status[['Partida Real','Partida Prevista','Diferença_segundos']].head(5)

,Partida Real,Partida Prevista,Diferença_segundos
0,2024-01-03 23:48:00,2024-01-03 23:55:00,-420.0
1,2024-01-03 03:38:00,2024-01-03 23:55:00,-73020.0
2,2024-01-03 00:09:00,2024-01-03 00:55:00,-2760.0
3,2024-01-03 21:58:00,2024-01-03 20:50:00,4080.0
4,2024-01-03 22:34:00,2024-01-03 22:30:00,240.0


Observamos que Diferença_segundos < 0 indicam voos adiantados e Diferença_segundos > 0, atrasados. Assim, vamos categorizá-los:

In [113]:
# Pontual = diferença da hora prevista para realizada de 5 minutos ou menos, seja adiantada ou atrasada
df_status.loc[df_status['Diferença_segundos'].abs() <= 300, 'Status_voo'] = 'Pontual'

# Atraso de 5 a 20 min:
df_status.loc[(df_status['Diferença_segundos'].abs() > 300) & (df_status['Diferença_segundos'].abs() <= 1200), 'Status_voo'] = 'De 5 a 20min atrasado/adiantado'

# Atraso de mais de 20min
df_status.loc[df_status['Diferença_segundos'].abs() > 1200, 'Status_voo'] = '20min+ atrasado/adiantado'

In [115]:
df_status['Status_voo'].value_counts(normalize=True).round(2)

Status_voo
De 5 a 20min atrasado/adiantado    0.45
Pontual                            0.41
20min+ atrasado/adiantado          0.14
Name: proportion, dtype: float64